In [1]:
!wget https://github.com/phatjkk/lora-in-llm/raw/main/bloomz-lora-vi-chatbot.zip

--2023-07-01 11:50:41--  https://github.com/phatjkk/lora-in-llm/raw/main/bloomz-lora-vi-chatbot.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/phatjkk/lora-in-llm/main/bloomz-lora-vi-chatbot.zip [following]
--2023-07-01 11:50:42--  https://raw.githubusercontent.com/phatjkk/lora-in-llm/main/bloomz-lora-vi-chatbot.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8650600 (8.2M) [application/zip]
Saving to: ‘bloomz-lora-vi-chatbot.zip’

bloomz-lora-vi-chat 100%[===================>]   8.25M  --.-KB/s    in 0.06s   

2023-07-01 11:50:42 (137 MB/s) - ‘bloomz-lora-vi-chatbot.zip’ saved [865

In [2]:
!unzip bloomz-lora-vi-chatbot.zip -d /content/bloomz-lora-vi-chatbot

Archive:  bloomz-lora-vi-chatbot.zip
   creating: /content/bloomz-lora-vi-chatbot/bloomz-lora-en-ecommerce/
  inflating: /content/bloomz-lora-vi-chatbot/bloomz-lora-en-ecommerce/tokenizer_config.json  
  inflating: /content/bloomz-lora-vi-chatbot/bloomz-lora-en-ecommerce/adapter_model.bin  
  inflating: /content/bloomz-lora-vi-chatbot/bloomz-lora-en-ecommerce/tokenizer.json  
  inflating: /content/bloomz-lora-vi-chatbot/bloomz-lora-en-ecommerce/README.md  
  inflating: /content/bloomz-lora-vi-chatbot/bloomz-lora-en-ecommerce/adapter_config.json  
  inflating: /content/bloomz-lora-vi-chatbot/bloomz-lora-en-ecommerce/special_tokens_map.json  


In [3]:
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 21.5 MB/s eta 0:00:00


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
model_path = "./bloomz-lora-vi-chatbot/bloomz-lora-en-ecommerce/"
config = PeftConfig.from_pretrained(model_path)

# load base LLM model and tokenizer
tokenizer_from_file = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model_from_file = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path).to("cuda")

In [12]:
!pip install flask flask_ngrok flask_cors  pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=5fba48d2e3f82c51c919b27e7858e6dfc373edcd2f949ffaab7c71f424a655af
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [18]:
from flask import Flask,jsonify, request
from flask_cors import CORS
from pyngrok import ngrok
import re

In [30]:
port_no = 5000

app = Flask(__name__)
ngrok.set_auth_token("2DKp381n48VDYpt5ruxGmNJAPhG_6bcFKtoFQ4TRqF8JvsDDn")
CORS(app,origins=['.*'])
public_url =  ngrok.connect(port_no).public_url


@app.route('/')
def home():
      return "hello"
@app.route("/q", methods=["POST"])
def returnJson():
    parameter_value = request.form.get('m')
    question3 = str(parameter_value)
    prompt = "Hỏi:"+ question3 + '''
        Đáp:'''
    prompt
    # In ra kết quả
    inputs = tokenizer_from_file( prompt, return_tensors="pt")
    # Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
    # biến này lên gpu bằng hàm .to("cuda")
    with torch.autocast("cuda"):
      data = {}
      outputs = model_from_file.generate(input_ids=inputs["input_ids"].to("cuda"),
                              max_new_tokens=100,
                              no_repeat_ngram_size=3,
                              num_beams=3,
                              num_return_sequences=1)

      answer = tokenizer_from_file.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

      answer = re.findall(r"(?<=Đáp: ).*", answer)[0]

      data = {"answer": answer}
    return jsonify(data)


print(f"To acces the Gloable link please click {public_url}")

app.run(port=port_no)

DEBUG:flask_cors.extension:Configuring CORS with resources: {'/*': {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}}


To acces the Gloable link please click https://defe-104-196-251-62.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
DEBUG:flask_cors.extension:Request to '/q' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
INFO:werkzeug:127.0.0.1 - - [01/Jul/2023 12:19:40] "POST /q HTTP/1.1" 200 -
DEBUG:flask_cors.extension:Request to '/q' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'suppor